In [1]:
import pandas as pd
import QUANTAXIS as qtx
from datetime import datetime, timedelta

In [2]:
TOP_STOCKS=500
CHECK_DAYS=30

In [3]:
def mtm_ind(data, col, N=12, M=6):
    mtm = (data[col] - qtx.REF(data[col], N)) / qtx.REF(data[col], N)
    mtmma = qtx.MA(mtm, M)
    return pd.DataFrame({'MTM': mtm, 'MTMMA': mtmma})

In [4]:
code_list = qtx.QA_fetch_stock_list()
finfo =  qtx.QA_fetch_stock_info(list(code_list.code))
fcnd = qtx.QA_fetch_stock_day_adv(list(code_list.code),
                                  start=(datetime.today()-timedelta(days=CHECK_DAYS+30)).strftime('%Y-%m-%d'), 
                                  end=datetime.today().strftime('%Y-%m-%d')).to_qfq()


In [5]:
fbase = fcnd.data.join(finfo.loc[finfo.ipo_date != 0, ['ipo_date','industry']], how='inner').reset_index()
fbase['ipodays'] = fbase.apply(lambda x: (x.date - datetime.strptime(str(x.ipo_date), '%Y%m%d')).days, axis=1)
fbase = fbase.set_index(['date', 'code']) 
fbase.head()

open   high    low  close     volume        amount  adj  \
date       code                                                               
2021-01-04 000001  19.10  19.10  18.44  18.60  1554216.0  2.891682e+09  1.0   
           000002  28.40  28.40  27.52  27.78  1468441.0  4.079417e+09  1.0   
           000004  20.69  21.19  20.52  21.02    27398.0  5.726255e+07  1.0   
           000005   2.53   2.53   2.51   2.52    48963.0  1.231635e+07  1.0   
           000006   5.64   5.64   5.50   5.52    81326.0  4.502973e+07  1.0   

                   ipo_date  industry  ipodays  
date       code                                 
2021-01-04 000001  19910403         1    10869  
           000002  19910129        11    10933  
           000004  19910114        24    10948  
           000005  19901210        39    10983  
           000006  19920427        11    10479

In [6]:
mtm=fcnd.add_func(mtm_ind, col='close', N=20).loc[:, ['MTM','MTMMA']]
mtm.head()

MTM  MTMMA
date       code              
2021-01-04 000001  NaN    NaN
           000002  NaN    NaN
           000004  NaN    NaN
           000005  NaN    NaN
           000006  NaN    NaN

In [7]:
smtm = fbase.loc[fbase.ipodays > 180, :].join(mtm)
smtm.head()

open   high    low  close     volume        amount  adj  \
date       code                                                               
2021-01-04 000001  19.10  19.10  18.44  18.60  1554216.0  2.891682e+09  1.0   
           000002  28.40  28.40  27.52  27.78  1468441.0  4.079417e+09  1.0   
           000004  20.69  21.19  20.52  21.02    27398.0  5.726255e+07  1.0   
           000005   2.53   2.53   2.51   2.52    48963.0  1.231635e+07  1.0   
           000006   5.64   5.64   5.50   5.52    81326.0  4.502973e+07  1.0   

                   ipo_date  industry  ipodays  MTM  MTMMA  
date       code                                             
2021-01-04 000001  19910403         1    10869  NaN    NaN  
           000002  19910129        11    10933  NaN    NaN  
           000004  19910114        24    10948  NaN    NaN  
           000005  19901210        39    10983  NaN    NaN  
           000006  19920427        11    10479  NaN    NaN

In [8]:
smtm['rank'] = smtm.groupby('date')['MTM'].rank(method='dense', ascending=False)
smtm.loc[smtm['rank']<TOP_STOCKS,:].head(10)

open   high    low  close     volume        amount  adj  \
date       code                                                               
2021-02-01 000001  23.00  24.99  22.70  24.55  1475239.0  3.529557e+09  1.0   
           000026  12.63  13.04  12.20  12.60   109580.0  1.368616e+08  1.0   
           000050  16.17  17.14  16.07  16.96   403545.0  6.807325e+08  1.0   
           000100   8.70   9.19   8.60   9.03  4870139.0  4.341706e+09  1.0   
           000155  15.56  15.71  14.25  14.42  1474046.0  2.197157e+09  1.0   
           000157  12.46  13.66  12.26  13.58  2185961.0  2.869445e+09  1.0   
           000301  14.21  14.65  12.22  14.01   743802.0  1.032744e+09  1.0   
           000338  21.50  22.49  20.90  21.88  1707071.0  3.716208e+09  1.0   
           000403  41.42  41.50  40.08  40.47    17611.0  7.157565e+07  1.0   
           000408   8.80   9.35   8.73   8.77   221833.0  1.980523e+08  1.0   

                   ipo_date  industry  ipodays       MTM  MTMMA   rank  
date       code                                                         
2021-02-01 000001  19910403         1    10897  0.319892    NaN   58.0  
           000026  19930603        12    10105  0.065089    NaN  465.0  
           000050  19950315        35     9455  0.140551    NaN  240.0  
           000100  20040130        35     6212  0.257660    NaN   98.0  
           000155  20000926        16     7433  0.682614    NaN    5.0  
           000157  20001012        26     7417  0.328767    NaN   51.0  
           000301  20000529        27     7553  0.519523    NaN   11.0  
           000338  20070430         7     5026  0.365793    NaN   41.0  
           000403  19960628        34     8984  0.099728    NaN  340.0  
           000408  19960628        28     8984  0.108723    NaN  311.0

In [9]:
raw = pd.read_csv("c:\\Users\\mUSicX\\Documents\\股票相关\\20200922SW.csv")
raw['c'] = raw.code.apply(lambda x: x[:6])
raw['mkt'] = raw.code.apply(lambda x: 0 if x.endswith('SZ') else 1)
raw['ind_cat1'] = raw.ind.apply(lambda x: x.split('--')[0])
raw['ind_cat2'] = raw.ind.apply(lambda x: x.split('--')[1].replace('Ⅱ', ''))
raw['ind_cat3'] = raw.ind.apply(lambda x: x.split('--')[2].replace('Ⅲ', ''))

raw['ind_cat'] = raw.apply(lambda x: x.ind_cat1+'-'+x.ind_cat2, axis=1)
raw[:10]

,code,ind,c,mkt,ind_cat1,ind_cat2,ind_cat3,ind_cat
0,000009.SZ,综合--综合Ⅱ--综合Ⅲ,000009,0,综合,综合,综合,综合-综合
1,000409.SZ,综合--综合Ⅱ--综合Ⅲ,000409,0,综合,综合,综合,综合-综合
2,000632.SZ,综合--综合Ⅱ--综合Ⅲ,000632,0,综合,综合,综合,综合-综合
3,000638.SZ,综合--综合Ⅱ--综合Ⅲ,000638,0,综合,综合,综合,综合-综合
4,000652.SZ,综合--综合Ⅱ--综合Ⅲ,000652,0,综合,综合,综合,综合-综合
5,000839.SZ,综合--综合Ⅱ--综合Ⅲ,000839,0,综合,综合,综合,综合-综合
6,002967.SZ,综合--综合Ⅱ--综合Ⅲ,002967,0,综合,综合,综合,综合-综合
7,300012.SZ,综合--综合Ⅱ--综合Ⅲ,300012,0,综合,综合,综合,综合-综合
8,300688.SZ,综合--综合Ⅱ--综合Ⅲ,300688,0,综合,综合,综合,综合-综合
9,300797.SZ,综合--综合Ⅱ--综合Ⅲ,300797,0,综合,综合,综合,综合-综合


In [9]:
from quantax import data_query
raw = data_query.local_get_stock_sw_block(code_list.code.tolist(), '2021-03-02')
raw['ind_cat'] = raw.apply(lambda x: x.sw1 + '-' + x.sw2, axis=1)
raw[:10]

type,sw1,sw2,sw3,sw,ind_cat
code,,,,,
000001,银行I,银行II,银行III,银行I-银行II-银行III,银行I-银行II
000002,房地产I,房地产开发II,房地产开发III,房地产I-房地产开发II-房地产开发III,房地产I-房地产开发II
000004,医药生物I,化学制药II,化学制剂III,医药生物I-化学制药II-化学制剂III,医药生物I-化学制药II
000005,公用事业I,环保工程及服务II,环保工程及服务III,公用事业I-环保工程及服务II-环保工程及服务III,公用事业I-环保工程及服务II
000006,房地产I,房地产开发II,房地产开发III,房地产I-房地产开发II-房地产开发III,房地产I-房地产开发II
000007,商业贸易I,商业物业经营II,一般物业经营III,商业贸易I-商业物业经营II-一般物业经营III,商业贸易I-商业物业经营II
000008,机械设备I,运输设备II,铁路设备III,机械设备I-运输设备II-铁路设备III,机械设备I-运输设备II
000009,综合I,综合II,综合III,综合I-综合II-综合III,综合I-综合II
000010,建筑装饰I,园林工程II,园林工程III,建筑装饰I-园林工程II-园林工程III,建筑装饰I-园林工程II


In [12]:
indict = raw['ind_cat'].to_dict()

#indict = industry['ind_cat']
#industry

In [13]:
smtm = smtm.reset_index()
smtm['cat'] = smtm.apply(lambda x: indict[str(x.code)], axis=1)
smtm = smtm.loc[:, ['date', 'code', 'cat', 'rank']]
smtm.head(10)

,date,code,cat,rank
0,2021-01-04,000001,银行I-银行II,NaN
1,2021-01-04,000002,房地产I-房地产开发II,NaN
2,2021-01-04,000004,医药生物I-化学制药II,NaN
3,2021-01-04,000005,公用事业I-环保工程及服务II,NaN
4,2021-01-04,000006,房地产I-房地产开发II,NaN
5,2021-01-04,000007,商业贸易I-商业物业经营II,NaN
6,2021-01-04,000008,机械设备I-运输设备II,NaN
7,2021-01-04,000009,综合I-综合II,NaN
8,2021-01-04,000010,建筑装饰I-园林工程II,NaN
9,2021-01-04,000011,房地产I-房地产开发II,NaN


In [14]:
tops = smtm.loc[smtm['rank']<TOP_STOCKS,:].groupby(['date', 'cat'])['code'].count()
bases = smtm.groupby(['date', 'cat'])['code'].count()
rps = pd.merge(tops.reset_index(), bases.reset_index(), on=['date', 'cat'])
rps.rename(columns={'code_x':'high', 'code_y':'full'}, inplace=True)
rps.head(10)

,date,cat,high,full
0,2021-02-01,交通运输I-物流II,7,40
1,2021-02-01,交通运输I-航空运输II,2,8
2,2021-02-01,休闲服务I-其他休闲服务II,1,3
3,2021-02-01,传媒I-互联网传媒II,10,58
4,2021-02-01,传媒I-文化传媒II,10,75
5,2021-02-01,传媒I-营销传播II,5,33
6,2021-02-01,公用事业I-燃气II,2,24
7,2021-02-01,公用事业I-环保工程及服务II,5,55
8,2021-02-01,公用事业I-电力II,5,75
9,2021-02-01,农林牧渔I-农产品加工II,3,17


In [15]:
rps['score'] = rps.high / rps.full * rps.high
rps['rank'] = rps.groupby('date')['score'].rank(ascending=False, method='dense', na_option='bottom')
rps['cap'] = rps.apply(lambda r: '{}/{}'.format(r.high, r.full), axis=1)
rps.head(10)

,date,cat,high,full,score,rank,cap
0,2021-02-01,交通运输I-物流II,7,40,1.225000,25.0,7/40
1,2021-02-01,交通运输I-航空运输II,2,8,0.500000,38.0,2/8
2,2021-02-01,休闲服务I-其他休闲服务II,1,3,0.333333,45.0,1/3
3,2021-02-01,传媒I-互联网传媒II,10,58,1.724138,17.0,10/58
4,2021-02-01,传媒I-文化传媒II,10,75,1.333333,23.0,10/75
5,2021-02-01,传媒I-营销传播II,5,33,0.757576,31.0,5/33
6,2021-02-01,公用事业I-燃气II,2,24,0.166667,53.0,2/24
7,2021-02-01,公用事业I-环保工程及服务II,5,55,0.454545,41.0,5/55
8,2021-02-01,公用事业I-电力II,5,75,0.333333,45.0,5/75
9,2021-02-01,农林牧渔I-农产品加工II,3,17,0.529412,37.0,3/17


In [16]:
rps = rps.loc[rps.date > (datetime.today()-timedelta(days=CHECK_DAYS)).strftime('%Y-%m-%d'), :]

In [17]:
ind_pivot = rps.pivot(index='cat', columns='date', values=['score','rank', 'cap'])
ind_pivot = ind_pivot.reindex(sorted(ind_pivot.columns, reverse=True), axis=1)
ind_pivot.fillna(0, inplace=True)
ind_pivot.sort_values(by=ind_pivot.columns[0], ascending=False, inplace=True)
top_ind = ind_pivot.loc[(ind_pivot.score > 1).any(axis=1),:]
top_ind

score                                              \
date             2021-03-04 2021-03-03 2021-03-02 2021-03-01 2021-02-26   
cat                                                                       
化工I-化学原料II         9.000000  11.111111  10.027778  12.250000  14.297297   
公用事业I-环保工程及服务II    8.491228   8.491228  11.859649   5.070175   4.413793   
钢铁I-钢铁II           8.257143   4.828571   0.714286   1.400000   0.735294   
化工I-化学制品II         8.004329   9.562771   7.703057   9.280702  10.577093   
有色金属I-工业金属II       6.113208   6.113208   2.716981   6.942308   8.480769   
休闲服务I-旅游综合II       5.333333   5.333333   4.083333   4.083333   4.083333   
机械设备I-专用设备II       4.687500   5.005208   4.712042   4.712042   2.534031   
休闲服务I-景点II         4.454545   5.818182   5.818182   5.818182   5.818182   
有色金属I-稀有金属II       4.166667   4.166667   1.500000   2.666667   2.560000   
有色金属I-金属非金属新材料II   4.000000   1.361111   0.250000   0.694444   0.444444   
机械设备I-通用机械II       3.278689   2.959016   2.959016   2.368852   2.959016   
休闲服务I-酒店II         2.666667   2.666667   2.666667   1.500000   2.666667   
化工I-化学纤维II         2.560000   3.240000   4.000000   4.000000   2.560000   
采掘I-采掘服务II         2.400000   0.266667   0.600000   0.600000   1.666667   
纺织服装I-服装家纺II       2.283019   2.283019   2.283019   1.886792   1.528302   
建筑材料I-其他建材II       1.883721   1.454545   1.840909   0.818182   0.204545   
采掘I-其他采掘II         1.600000   0.900000   0.100000   0.900000   0.400000   
轻工制造I-家用轻工II       1.428571   0.700000   1.728571   1.157143   1.428571   
建筑材料I-水泥制造II       1.388889   2.000000   1.388889   0.888889   1.388889   
农林牧渔I-林业II         1.333333   0.333333   0.333333   0.000000   0.000000   
采掘I-煤炭开采II         1.324324   0.243243   0.243243   0.432432   0.432432   
房地产I-房地产开发II       1.252174   1.252174   1.252174   1.252174   2.226087   
轻工制造I-造纸II         1.250000   1.800000   2.450000   3.200000   1.800000   
汽车I-汽车零部件II        1.224638   1.630435   1.868613   0.729927   0.729927   
综合I-综合II           1.125000   0.781250   1.531250   0.781250   1.531250   
交通运输I-航空运输II       1.125000   1.125000   1.125000   2.000000   2.000000   
医药生物I-医疗器械II       1.094595   0.486486   0.337838   0.486486   0.216216   
电子I-光学光电子II        1.051948   1.051948   1.298701   0.467532   1.282051   
医药生物I-医疗服务II       1.041667   1.041667   1.041667   0.166667   0.166667   
休闲服务I-其他休闲服务II     1.000000   1.000000   2.250000   2.250000   2.250000   
建筑装饰I-装修装饰II       0.925926   0.592593   0.925926   1.333333   0.925926   
有色金属I-黄金II         0.818182   0.363636   0.090909   0.363636   0.818182   
家用电器I-视听器材II       0.818182   0.818182   0.818182   1.454545   0.818182   
非银金融I-多元金融II       0.806452   1.161290   1.161290   1.161290   0.516129   
电气设备I-电源设备II       0.761905   0.433735   0.433735   0.301205   0.195122   
商业贸易I-专业零售II       0.750000   0.750000   1.333333   1.333333   1.454545   
电气设备I-电气自动化设备II    0.543478   0.086957   0.086957   0.086957   0.088889   
建筑装饰I-专业工程II       0.533333   0.833333   1.200000   0.533333   0.833333   
电子I-电子制造II         0.529412   0.720588   0.941176   0.941176   0.529412   
建筑材料I-玻璃制造II       0.500000   1.125000   1.125000   1.125000   0.500000   
医药生物I-化学制药II       0.445455   0.736364   1.309091   0.917431   0.917431   
轻工制造I-包装印刷II       0.410256   0.410256   0.230769   0.641026   0.923077   
汽车I-汽车整车II         0.409091   0.428571   0.727273   0.181818   0.181818   
计算机I-计算机应用II       0.323232   0.727273   0.989899   1.292929   0.611111   
传媒I-互联网传媒II        0.275862   1.103448   2.526316   2.913793   2.482759   
农林牧渔I-饲料II         0.266667   0.600000   0.600000   2.400000   2.400000   
银行I-银行II           0.250000   0.250000   0.111111   0.111111   0.694444   
交通运输I-物流II         0.219512   0.097561   0.219512   0.219512   0.609756   
传媒I-文化传媒II         0.213333   0.333333   0.480000   1.080000   1.333333   
化工I-橡胶II           0.210526   0.052632   0.210526   0.842105   0.473684   
化工I-石油化工II         0.210526   0.473684   0.4736

In [18]:
top_ind.to_excel('c:\\Users\\mUSicX\\Desktop\\block_rps_scores{}.xls'.format(TOP_STOCKS))

In [ ]:
mtest = mtm.reset_index()
mtest.loc[mtest.code=='300758',:]